Quantum Circuits

Quantum computers can only use a specific set of gates (universal gate set). Given the entanglers and their amplitudes found in Step 3, one can find corresponding representation of these operators in terms of elementary gates using the following procedure.


In [1]:
import tequila as tq
from utility import *

In [3]:
H = tq.QubitHamiltonian.from_openfermion(get_qubit_hamiltonian('h2', 2, 'sto-3g', qubit_transf='jw'))

a = tq.Variable("tau_0")
U = construct_QMF_ansatz(4)
U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string("X(0)Y(1)X(2)X(3)"), angle=a)
print(U)

circuit: 
Rx(target=(0,), parameter=beta_0)
Rz(target=(0,), parameter=gamma_0)
Rx(target=(1,), parameter=beta_1)
Rz(target=(1,), parameter=gamma_1)
Rx(target=(2,), parameter=beta_2)
Rz(target=(2,), parameter=gamma_2)
Rx(target=(3,), parameter=beta_3)
Rz(target=(3,), parameter=gamma_3)
Exp-Pauli(target=(0, 1, 2, 3), control=(), parameter=tau_0, paulistring=X(0)Y(1)X(2)X(3))



One can check the expectation value to see it is near the ground state energy.

In [4]:
E = tq.ExpectationValue(H=H, U=U)
vars = {'beta_1': 3.141592624143881, 'beta_0': 3.141592624143881, 'tau_0': 1.1331410014096885, 'gamma_1': 0.0, 'beta_3': 0.0, 'gamma_3': 0.0, 'gamma_2': 0.0, 'gamma_0': 0.0, 'beta_2': 0.0} # values obtained from step 3
print(tq.simulate(E, variables=vars))

-0.9486411121761625


In [5]:
from qiskit import IBMQ
IBMQ.save_account('47127133da8013cd8352510919c5df452c78eb470707ca21a19dd1acecc31fdf0df0b98d4bf187ddae420c658e55c52f85ac3fe9f27f06acd1fbff32c22590ec')

configrc.store_credentials:WARNING:1970-01-01 11:16:02,706: Credentials already present. Set overwrite=True to overwrite.


In [12]:
# list of devices available can be found in ibmq account page
tq.simulate(E, variables=vars, samples=100, backend="qiskit", device='ibmq_qasm_simulator')

-0.9386970787051225

##Error mitigation protocol:

Here we want to correct for violations of the particle conservation number introduced by QCC.
Following ref[14] as suggested in the instructions we undertand that what needs to be done is to identify solutions that violate this symmetry and discard them. Given the decomposition of the total H into fragments :<Psi|H|Psi>=<Psi|A|Psi> +<Psi|B|Psi> = \sum_n a_n |<Psi|f_n>|^2 + \sum_n b_n |<Psi|g_n>|^2 where A and B are unitaries that commute and n is the label of the measurement(sample). If the projector of the number of particles, P_N=|N><N|, acting on f_n(g_n) gives zero then we discard a_n(b_n). Thus if  P_N|f_n> =0 or P_N|g_n>=0, then we get rid of the measurements a_n/b_n and re-compute <Psi|H|Psi> without them. 
We need to perform the following steps to achieve the goal:
1. Write the two commutative unitary operators obtained through the QWC procedure in task 4 as a matrix and compute their eigenvectors f_n and g_n.
2. Write the number operator N as a matrix operator in the qubit space, compute the eigenvectors and eigenvalues . Select the eigenvalues corresponding to the particle number 2 and construct the projector as \sum_i |n_i><n_i| where i only includes n_i whose eigenvalue is 2.
3. Simulate a_n |<Psi|f_n>|^2 and  b_n |<Psi|g_n>|^2 and for each measurement n compute P_N|f_n> =0 and P_N|g_n>. If it gives zero discard  a_n/b_n.
4. Compute <Psi|H|Psi> including first all the measurements and then discarding the ones that violate conservation of particles. Compare the total energy and the expectation value of N in both cases. Did the error mitigation protocol improve the accuracy? Did it enforce the conservation of particles?

3.Simulate <A> and <B> 

NameError: name 'allzA' is not defined

1. Write A and B taken from task 4 as matrices and compute eigenvectors.

In [22]:
X = np.array([[0.,  1.], [1.,  0.]])
Y = np.array([[0., -1j], [1j,  0.]])
Z = np.array([[1.,  0.], [0., -1.]])
I = np.array([[1.,  0.], [0., 1.]])

A=-0.3276081896748086 [] +
0.15542669077992818 [Z0] +
0.15660062488237944 [Z0 Z1] +
0.04919764587136754 [Z0 Z1 Z3] +
0.1062290449085607 [Z0 Z2] +
-0.04919764587136754 [Z0 Z3] +
0.1062290449085607 [Z1] +
0.15542669077992818 [Z1 Z2] +
-0.04919764587136754 [Z1 Z2 Z3] +
0.16326768673564326 [Z2] +
0.04919764587136754 [Z2 Z3]

------------------------------------

B=0.13716572937099472 [Z0] +
0.13716572937099472 [Z1] +
-0.13036292057109117 [Z2] +
-0.13036292057109117 [Z3]

In [44]:
# Building  A for each qubit
#I, X, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[1, 0], [0, -1]])
A_qubit0 = -0.3276081896748086 * I + 0.15542669077992818 * Z +0.15660062488237944*Z 
+ 0.04919764587136754*Z+0.1062290449085607*Z -0.04919764587136754*Z
A_qubit1 = -0.3276081896748086 * I + (0.15660062488237944+0.04919764587136754+0.1062290449085607+0.15542669077992818-0.04919764587136754)*Z
A_qubit2 = -0.3276081896748086 * I + (0.1062290449085607+0.15542669077992818-0.04919764587136754+0.16326768673564326+0.04919764587136754)*Z
A_qubit3 = -0.3276081896748086 * I + (0.04919764587136754-0.04919764587136754-0.04919764587136754+0.04919764587136754)*Z
#print('qubit0:',A_qubit0)


In [52]:
#Construct the A matrix for the 4 qubits:
    
A01=np.kron(A_qubit0,A_qubit1)
A23=np.kron(A_qubit2, A_qubit3)
A=np.kron(A01,A23)

# Obtain the eigenvalues and eigenvectors

eigvals, eigenvectors_ = np.linalg.eigh(A)
print("\nThe eigenvalues in the A_matrix: \n {}".format(eigvals))
print("\nThe eigenvectors in the A_matrix: \n {}".format(eigenvectors))


The eigenvalues in the A_matrix: 
 [-1.52099611e-02 -1.52099611e-02 -1.42945704e-02 -1.42945704e-02
 -3.70499271e-04 -3.70499271e-04 -3.48201277e-04 -3.48201277e-04
  4.50283918e-05  4.50283918e-05  1.84853290e-03  1.84853290e-03
  2.86504390e-03  2.86504390e-03  1.17617523e-01  1.17617523e-01]

The eigenvectors in the A_matrix: 
 [[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0

In [46]:
# Building B for each qubit
#I, X, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[1, 0], [0, -1]])
B_qubit0 = 0.13716572937099472*Z
B_qubit1 = 0.13716572937099472*Z
B_qubit2 = -0.13036292057109117*Z
B_qubit3 = -0.13036292057109117*Z

In [53]:
#Construct the B matrix for the 4 qubits:
    
B01=np.kron(B_qubit0,B_qubit1)
B23=np.kron(B_qubit2, B_qubit3)
B=np.kron(B01,B23)

# Obtain the eigenvalues and eigenvectors of B
#eigvals, _ = np.linalg.eigh(B)
eigvals, eigenvectors  = np.linalg.eigh(B)
print("\nThe eigenvalues in the B_matrix: \n {}".format(eigvals))
print("\nThe eigenvectors in the B_matrix: \n {}".format(eigenvectors))


The eigenvalues in the B_matrix: 
 [-0.00031974 -0.00031974 -0.00031974 -0.00031974 -0.00031974 -0.00031974
 -0.00031974 -0.00031974  0.00031974  0.00031974  0.00031974  0.00031974
  0.00031974  0.00031974  0.00031974  0.00031974]

The eigenvectors in the B_matrix: 
 [[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 

In [62]:
qubit_transf = 'jw' # Jordan-Wigner transformations
h2 = get_qubit_hamiltonian(mol='h2', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print(h2)

-0.3276081896748089 [] +
-0.04919764587136759 [X0 X1 Y2 Y3] +
0.04919764587136759 [X0 Y1 Y2 X3] +
0.04919764587136759 [Y0 X1 X2 Y3] +
-0.04919764587136759 [Y0 Y1 X2 X3] +
0.13716572937099494 [Z0] +
0.15660062488237958 [Z0 Z1] +
0.10622904490856085 [Z0 Z2] +
0.15542669077992843 [Z0 Z3] +
0.13716572937099494 [Z1] +
0.15542669077992843 [Z1 Z2] +
0.10622904490856085 [Z1 Z3] +
-0.1303629205710914 [Z2] +
0.1632676867356435 [Z2 Z3] +
-0.13036292057109133 [Z3]


In [ ]:
#Construct the N matrix and compute eigenvals and eigenvectors.
#N is taken from eq 6 in ref [14].

In [ ]:
# Defining pauli matrices 
I, X, Y, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[0, -1j], [1j, 0]]), np.array([[1, 0], [0, -1]])

In [8]:
Z0=tq.paulis.Z(0)
Z1=tq.paulis.Z(1)
Z2=tq.paulis.Z(2)
Z3=tq.paulis.Z(3)
Nop=2-0.5*(Z0 +Z1*Z0+Z2+Z3*Z2*Z1)
print(Nop)

-0.5000Z(0)-0.5000Z(0)Z(1)-0.5000Z(2)-0.5000Z(1)Z(2)Z(3)+2.0000


In [73]:
Nop= -0.13036292057109133 [Z3]
#2.0*Z0-0.5*(Z0+Z1*Z0)

TypeError: 'float' object is not subscriptable

In [71]:

# Build matrix representiation of the operator N
#n_qubits = openfermion.count_qubits(h2)
n_qubits=4
N_matrix = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex)
for term, term_coeff in Nop.terms.items(): # Iterate over pauli-words of N
    term = dict(term) # Dict[qubit_index, 'X'/'Y'/'Z']
    
    # Build matrix rep of current pauli-word using kronecker product to represent x_i y_j ...
    pw_matrix = np.identity(1)
    for i in range(n_qubits):
        if i not in term:        pw_matrix = np.kron(pw_matrix, I)
        else:
            if term[i] == 'X':   pw_matrix = np.kron(pw_matrix, X)
            elif term[i] == 'Y': pw_matrix = np.kron(pw_matrix, Y)
            else:                pw_matrix = np.kron(pw_matrix, Z)
    N_matrix += pw_matrix * term_coeff

eigvals, _ = np.linalg.eigh(N_matrix)
#print("The ground state energy from S1: ")
#obtain_PES('h2', [1], 'sto-3g', 'fci')
print("\nThe eigenvalues in the matrix representation of N: \n{}".format(eigvals))

AttributeError: 'QubitHamiltonian' object has no attribute 'terms'

In [57]:
#Matrix form of N
N_qubit0 = 2.0*I-0.5*(2.0*Z)
N_qubit1 = 2.0*I-0.5*(2.0)*Z
N_qubit2 =2.0*I -0.5*(2.0)*Z
N_qubit3 = 2.0*I-0.5*Z

In [58]:
#Construct the N matrix for the 4 qubits:
    
N01=np.kron(N_qubit0,N_qubit1)
N23=np.kron(N_qubit2, N_qubit3)
N=np.kron(N01,N23)

# Obtain the eigenvalues and eigenvectors of N
#eigvals, _ = np.linalg.eigh(B)
eigvals, eigenvectors  = np.linalg.eigh(N)
print("\nThe eigenvalues in the N_matrix: \n {}".format(eigvals))
print("\nThe eigenvectors in the B_matrix: \n {}".format(eigenvectors))


The eigenvalues in the N_matrix: 
 [ 1.5  2.5  4.5  4.5  4.5  7.5  7.5  7.5 13.5 13.5 13.5 22.5 22.5 22.5
 40.5 67.5]

The eigenvectors in the B_matrix: 
 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [9]:
Ne=tq.ExpectationValue(H=Nop,U=U)
print(Ne)

Objective with 1 unique expectation values
total measurements = 1
variables          = 9
types              = not compiled


In [10]:
print(tq.simulate(Ne,variables = vars))

1.7119084561361877
